In [28]:
# Load Libraries # 

import os
import pandas as pd 
import numpy as np 

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
## Create a list of all Lakers game files. (ie. files where the naem contains LAL) #

# Here, my path is : "C:/Users/mmiric/Documents/NBA Data/1718 NBA Data/TRAIN", but yours 
# might differ #

TeamNames = []

for file in os.listdir("1718/TRAIN"):
    if "@" in file:
        TeamNames.append(str(file[file.find('@')-3:file.find('@')]))
        TeamNames.append(str(file[file.find('@')+1:file.find('@')+4]))

TeamNames = set(TeamNames)

In [100]:
# Let's see the head (first 5 rows) in the first data file: 

# Directory = "1718/TRAIN/"

# data = pd.read_csv(Directory + LAL_Files[0])

#pd.to_timedelta(data['play_length'])

def countSecond(strtime):
    '''Caculate how many second each event'''
    try:
        numtime = pd.to_timedelta(strtime)
        return numtime.total_seconds()
    except:
        return 0

In [ ]:
# Create playerPlusMinus total dataset to accept and record each player's +/- in each Game
Directory = "1718/TRAIN/"
playerPlusMinus = pd.DataFrame()

for TeamName in TeamNames:
    print(TeamName)
    for file in os.listdir("1718/TRAIN"):

        if "@"+TeamName in file: 
            #print(file)
            data = pd.read_csv(Directory + file)

            HomeAway = "Home"

            temp_data = data[['h1', 'h2', 'h3', 'h4', 'h5', 'team', 'points', 'play_length']]

            temp_data['ScoredConceded'] = np.where(temp_data['team'] == TeamName, "Scored", "Conceded")    
            temp_data = temp_data.loc[temp_data['points'].notnull()]
            temp_data['play_length'] = temp_data['play_length'].apply(countSecond)
            
            temp_data2 = pd.DataFrame([])

            i = 0
            while i < 5:
                i += 1
                hname = 'h'+str(i)
                tmp = temp_data[[hname, 'team', 'points', 'ScoredConceded', 'play_length']]
                tmp.rename(columns={hname:'player'}, inplace=True)
                tmp['position'] = hname
                tmp = tmp.loc[tmp['points']!=0]

                temp_data2 = temp_data2.append(tmp)

            # Calculate for each game, players score and conceded points    
            temp_data3 = pd.pivot_table(temp_data2, index = ['player', 'ScoredConceded'],\
                                        values = ['points','play_length'], aggfunc = np.sum).unstack(level=1)
            #print(temp_data2)
            

            # Make the pivot_table into a better format
            temp_data3 = temp_data3.reset_index()

            temp_data3.columns = ['player', 'ConcededSecond', 'ScoredSecond', 'Conceded', 'Scored']

            temp_data3['PlusMinus'] = np.nan_to_num(temp_data3['Scored']) - np.nan_to_num(temp_data3['Conceded'])
            temp_data3['Team'] = TeamName
            temp_data3['GameID'] = data['game_id']
            temp_data3['HomeAway'] = HomeAway
            temp_data3['Seconds'] = np.nan_to_num(temp_data3['ConcededSecond']) + np.nan_to_num(temp_data3['ScoredSecond'])

            # After processing the entire file, append player's +/- into a whole dataset    
            playerPlusMinus = playerPlusMinus.append(temp_data3.reset_index(), ignore_index=False)
            
        elif TeamName+"@" in file: 
            #print(file)
            data = pd.read_csv(Directory + file)

            HomeAway = "Away"

            temp_data = data[['a1', 'a2', 'a3', 'a4', 'a5', 'team', 'points', 'play_length']]

            temp_data['ScoredConceded'] = np.where(temp_data['team'] == TeamName, "Scored", "Conceded")    
            temp_data = temp_data.loc[temp_data['points'].notnull()]
            temp_data['play_length'] = temp_data['play_length'].apply(countSecond)
            
            temp_data2 = pd.DataFrame([])

            i = 0
            while i < 5:
                i += 1
                hname = 'a'+str(i)
                tmp = temp_data[[hname, 'team', 'points', 'ScoredConceded', 'play_length']]
                tmp.rename(columns={hname:'player'}, inplace=True)
                tmp['position'] = hname
                tmp = tmp.loc[tmp['points']!=0]

                temp_data2 = temp_data2.append(tmp)

            # Calculate for each game, players score and conceded points    
            temp_data3 = pd.pivot_table(temp_data2, index = ['player', 'ScoredConceded'],\
                                        values = ['points','play_length'], aggfunc = np.sum).unstack(level=1)
            
            # Make the pivot_table into a better format
            temp_data3 = temp_data3.reset_index()
            temp_data3.columns = ['player', 'ConcededSecond', 'ScoredSecond', 'Conceded', 'Scored']

            temp_data3['PlusMinus'] = np.nan_to_num(temp_data3['Scored']) - np.nan_to_num(temp_data3['Conceded'])
            temp_data3['Team'] = TeamName
            temp_data3['GameID'] = data['game_id']
            temp_data3['HomeAway'] = HomeAway
            temp_data3['Seconds'] = np.nan_to_num(temp_data3['ConcededSecond']) + np.nan_to_num(temp_data3['ScoredSecond'])
            #print(temp_data3)

            # After processing the entire file, append player's +/- into a whole dataset    
            playerPlusMinus = playerPlusMinus.append(temp_data3.reset_index(), ignore_index=False)
            #print(playerPlusMinus)

    playerPlusMinus.groupby(['player', 'HomeAway'])\
    .agg({'PlusMinus':['mean','std'], 'GameID':'count', 'Seconds':'mean'})\
    .to_csv('Player Plus Minus'+TeamName+'.csv')


LAL
ORL
POR
MEM
CHA
MIL
MIA
CHI
BOS
GSW
NYK
ATL
DAL
OKC
TOR
DET
LAC
MIN
SAS
IND
PHX
HOU
WAS
UTA
NOP
SAC
PHI
BKN
DEN


In [9]:
# Directory = "1718/TRAIN/"
# file =  LAL_Files[0]

# data = pd.read_csv(Directory + file)

# # DEFINE IF HOME OR AWAY GAME # 

# print(file)

LAL
                         play_length          points       
ScoredConceded              Conceded Scored Conceded Scored
player                                                     
Brandon Ingram                 351.0  284.0     79.0   84.0
Brook Lopez                    290.0  269.0     64.0   72.0
Corey Brewer                   123.0  140.0     32.0   27.0
Jordan Clarkson                190.0  210.0     42.0   44.0
Josh Hart                      117.0  133.0     30.0   26.0
Julius Randle                  156.0  148.0     38.0   35.0
Kentavious Caldwell-Pope       327.0  277.0     72.0   80.0
Kyle Kuzma                     355.0  351.0     83.0   88.0
Lonzo Ball                     321.0  273.0     70.0   79.0

In [31]:
# Create playerPlusMinus total dataset to accept and record each player's +/- in each Game
# if the team is at home play

# playerPlusMinus = pd.DataFrame([])

# for file in os.listdir("1718/TRAIN"):
    
#     if "@CLE" in file: 
#         print(file)
#         data = pd.read_csv(Directory + file)

#         HomeAway = "Home"

#         temp_data = data[['h1', 'h2', 'h3', 'h4', 'h5', 'team', 'points']]

#         temp_data['ScoredConceded'] = np.where(temp_data['team'] == "CLE", "Scored", "Conceded")    
#         temp_data = temp_data.loc[temp_data['points'].notnull()]
#         temp_data2 = pd.DataFrame([])

#         i = 0
#         while i < 5:
#             i += 1
#             hname = 'h'+str(i)
#             tmp = temp_data[[hname, 'team', 'points', 'ScoredConceded']]
#             tmp.rename(columns={hname:'player'}, inplace=True)
#             tmp['position'] = hname
#             tmp = tmp.loc[tmp['points']!=0]

#             temp_data2 = temp_data2.append(tmp)

#         # Calculate for each game, players score and conceded points    
#         temp_data3 = pd.pivot_table(temp_data2, index = ['player', 'ScoredConceded'], values = ['points'], aggfunc = np.sum).unstack(level=1)

#         # Make the pivot_table into a better format
#         temp_data3 = temp_data3.reset_index()
#         temp_data3.columns = ['player', 'Conceded', 'Scored']

#         temp_data3['PlusMinus'] = np.nan_to_num(temp_data3['Scored']) - np.nan_to_num(temp_data3['Conceded'])
#         temp_data3['Team'] = 'CLE'
#         temp_data3['GameID'] = data['game_id']
#         temp_data3['HomeAway'] = HomeAway

#         # After processing the entire file, append player's +/- into a whole dataset    
#         playerPlusMinus = playerPlusMinus.append(temp_data3.reset_index(), ignore_index=False)


[2017-11-28]-0021700298-MIA@CLE.csv
[2018-03-21]-0021701065-TOR@CLE.csv
[2017-12-21]-0021700466-CHI@CLE.csv
[2018-01-26]-0021700715-IND@CLE.csv
[2018-01-31]-0021700757-MIA@CLE.csv
[2018-01-28]-0021700734-DET@CLE.csv
[2018-03-05]-0021700950-DET@CLE.csv
[2017-12-12]-0021700397-ATL@CLE.csv
[2018-02-03]-0021700779-HOU@CLE.csv
[2018-05-05]-0041700203-TOR@CLE.csv
[2018-03-19]-0021701050-MIL@CLE.csv
[2018-05-19]-0041700303-BOS@CLE.csv
[2018-04-11]-0021701220-NYK@CLE.csv
[2017-12-06]-0021700353-SAC@CLE.csv
[2018-04-05]-0021701176-WAS@CLE.csv
[2018-06-08]-0041700404-GSW@CLE.csv
[2018-04-29]-0041700137-IND@CLE.csv
[2018-02-27]-0021700908-BKN@CLE.csv
[2018-05-21]-0041700304-BOS@CLE.csv
[2018-04-18]-0041700132-IND@CLE.csv
[2017-11-01]-0021700106-IND@CLE.csv
[2018-01-15]-0021700646-GSW@CLE.csv
[2018-04-03]-0021701156-TOR@CLE.csv
[2018-02-25]-0021700894-SAS@CLE.csv
[2017-11-22]-0021700254-BKN@CLE.csv
[2018-03-01]-0021700923-PHI@CLE.csv
[2017-12-02]-0021700329-MEM@CLE.csv
[2018-05-07]-0041700204-TOR@

In [67]:
playerPlusMinus.groupby('player')['PlusMinus'].mean()

player
Aaron Brooks           -2.812500
Aaron Gordon            0.400000
Aaron Harrison         -7.250000
Aaron Jackson          -1.250000
Abdel Nader            -0.692308
Adreian Payne          -1.500000
Al Horford              3.950000
Al Jefferson           -0.714286
Al-Farouq Aminu         2.718750
Alan Williams           2.000000
Alec Burks              0.157895
Alec Peters             0.181818
Alex Abrines            2.085714
Alex Caruso             0.857143
Alex Len               -5.843750
Alex Poythress         -3.625000
Alfonzo McKinnie       -0.500000
Allen Crabbe           -0.833333
Amir Johnson            2.277778
Andre Drummond          2.684211
Andre Iguodala          5.894737
Andre Ingram           -4.000000
Andre Roberson          8.058824
Andrew Bogut           -1.818182
Andrew Harrison        -6.620690
Andrew White III       -7.500000
Andrew Wiggins          4.406250
Ante Zizic             -0.058824
Anthony Brown           3.000000
Anthony Davis           5.658537
   

In [66]:
playerPlusMinus.head(10)

,index,player,Conceded,Scored,PlusMinus,Team,GameID,HomeAway
0,0,Brandon Paul,9.0,12.0,3.0,SAS,"=""0021700126""",Home
1,1,Bryn Forbes,49.0,66.0,17.0,SAS,"=""0021700126""",Home
2,2,Danny Green,63.0,48.0,-15.0,SAS,"=""0021700126""",Home
3,3,Dejounte Murray,30.0,42.0,12.0,SAS,"=""0021700126""",Home
4,4,Kyle Anderson,51.0,41.0,-10.0,SAS,"=""0021700126""",Home
5,5,LaMarcus Aldridge,77.0,68.0,-9.0,SAS,"=""0021700126""",Home
6,6,Manu Ginobili,55.0,68.0,13.0,SAS,"=""0021700126""",Home
7,7,Patty Mills,71.0,66.0,-5.0,SAS,"=""0021700126""",Home
8,8,Pau Gasol,50.0,60.0,10.0,SAS,"=""0021700126""",Home
9,9,Rudy Gay,50.0,69.0,19.0,SAS,"=""0021700126""",Home


In [34]:
# Create playerPlusMinus total dataset to accept and record each player's +/- in each Game
# if the team is at away play

# playerPlusMinus = pd.DataFrame([])


for file in os.listdir("1718/TRAIN"):
    
    if "CLE@" in file: 
        print(file)
        data = pd.read_csv(Directory + file)

        HomeAway = "Away"

        temp_data = data[['a1', 'a2', 'a3', 'a4', 'a5', 'team', 'points']]

        temp_data['ScoredConceded'] = np.where(temp_data['team'] == "CLE", "Scored", "Conceded")    
        temp_data = temp_data.loc[temp_data['points'].notnull()]
        temp_data2 = pd.DataFrame([])

        i = 0
        while i < 5:
            i += 1
            hname = 'a'+str(i)
            tmp = temp_data[[hname, 'team', 'points', 'ScoredConceded']]
            tmp.rename(columns={hname:'player'}, inplace=True)
            tmp['position'] = hname
            tmp = tmp.loc[tmp['points']!=0]

            temp_data2 = temp_data2.append(tmp)

        # Calculate for each game, players score and conceded points    
        temp_data3 = pd.pivot_table(temp_data2, index = ['player', 'ScoredConceded'], values = ['points'], aggfunc = np.sum).unstack(level=1)

        # Make the pivot_table into a better format
        temp_data3 = temp_data3.reset_index()
        temp_data3.columns = ['player', 'Conceded', 'Scored']

        temp_data3['PlusMinus'] = np.nan_to_num(temp_data3['Scored']) - np.nan_to_num(temp_data3['Conceded'])
        temp_data3['Team'] = 'CLE'
        temp_data3['GameID'] = data['game_id']
        temp_data3['HomeAway'] = HomeAway

        # After processing the entire file, append player's +/- into a whole dataset    
        playerPlusMinus = playerPlusMinus.append(temp_data3.reset_index(), ignore_index=False)


[2017-10-25]-0021700058-CLE@BKN.csv
[2018-03-13]-0021701014-CLE@PHX.csv
[2017-11-11]-0021700181-CLE@DAL.csv
[2018-03-07]-0021700970-CLE@DEN.csv
[2018-01-03]-0021700559-CLE@BOS.csv
[2018-01-23]-0021700698-CLE@SAS.csv
[2018-03-27]-0021701110-CLE@MIA.csv
[2017-12-25]-0021700494-CLE@GSW.csv
[2018-05-13]-0041700301-CLE@BOS.csv
[2017-12-04]-0021700344-CLE@CHI.csv
[2018-05-23]-0041700305-CLE@BOS.csv
[2018-05-03]-0041700202-CLE@TOR.csv
[2018-02-11]-0021700835-CLE@BOS.csv
[2018-04-06]-0021701183-CLE@PHI.csv
[2018-02-23]-0021700878-CLE@MEM.csv
[2017-12-08]-0021700369-CLE@IND.csv
[2017-11-20]-0021700242-CLE@DET.csv
[2018-06-03]-0041700402-CLE@GSW.csv
[2017-11-27]-0021700291-CLE@PHI.csv
[2018-02-13]-0021700850-CLE@OKC.csv
[2018-05-15]-0041700302-CLE@BOS.csv
[2018-04-09]-0021701207-CLE@NYK.csv
[2017-12-30]-0021700534-CLE@UTA.csv
[2018-01-30]-0021700747-CLE@DET.csv
[2018-03-28]-0021701116-CLE@CHA.csv
[2018-01-12]-0021700619-CLE@IND.csv
[2017-12-27]-0021700513-CLE@SAC.csv
[2018-03-25]-0021701094-CLE@

In [35]:
playerPlusMinus.head(3)

,index,player,Conceded,Scored,PlusMinus,Team,GameID,HomeAway
0,0,Ante Zizic,4.0,NaN,-4.0,CLE,"=""0021700298""",Home
1,1,Cedi Osman,4.0,NaN,-4.0,CLE,"=""0021700298""",Home
2,2,Channing Frye,44.0,33.0,-11.0,CLE,"=""0021700298""",Home


In [13]:
playerPlusMinus.groupby(['player', 'HomeAway']).agg({'PlusMinus':['mean','std'], 'GameID':'count'})\
.to_csv('Player Plus Minus'+TeamName+'.csv')

In [10]:
playerPlusMinus.to_csv('Player Plus Minus'+TeamName+'.csv')

In [137]:
playerPlusMinus.groupby(['player', 'HomeAway'])\
.agg({'PlusMinus':['mean','std'], 'Team':'first', 'GameID':'count', 'Seconds':'mean'}).head(10)

PlusMinus             Team GameID     Seconds
                             mean        std first  count        mean
player         HomeAway                                              
Aaron Gordon   Away     -2.863636  12.139253   ORL     22  654.636364
               Home      0.400000  11.814539   ORL     25  606.280000
Aaron Harrison Away     -8.200000   6.496153   DAL      5  445.800000
               Home     -7.250000  14.056434   DAL      4  544.500000
Abdel Nader    Away     -3.478261   6.754591   BOS     23  157.695652
               Home     -0.692308   7.877915   BOS     26  162.307692
Adreian Payne  Away     -3.666667  15.821926   ORL      3  169.666667
               Home     -1.500000   0.707107   ORL      2  113.000000
Al Horford     Away      2.947368  11.467810   BOS     38  599.447368
               Home      3.950000  10.570763   BOS     40  610.900000

In [37]:
pd.to_datetime(data.loc['point]']>0]['play_length'])

SyntaxError: invalid syntax (<ipython-input-37-2e856b8df840>, line 1)

In [96]:
#pd.to_timedelta(data['play_length'])

def countSecond(strtime):
    try:
        numtime = pd.to_timedelta(strtime)
        #print(numtime.total_seconds())
        return numtime.total_seconds()
    except:
        return 0

In [97]:
countSecond('0:00:10')

10.0

In [136]:
#pd.to_timedelta('0:10:10').total_seconds()
tmp.play_length.apply(countSecond)

4      1.200000e-08
7      9.000000e-09
15     0.000000e+00
25     1.700000e-08
30     1.000000e-08
31     2.400000e-08
42     8.000000e-09
50     1.300000e-08
51     1.400000e-08
57     1.100000e-08
65     0.000000e+00
69     8.000000e-09
72     2.000000e-09
81     7.000000e-09
89     8.000000e-09
91     0.000000e+00
93     0.000000e+00
100    1.800000e-08
124    8.000000e-09
127    0.000000e+00
138    5.000000e-09
143    1.200000e-08
157    3.000000e-09
164    1.200000e-08
165    1.800000e-08
172    0.000000e+00
173    0.000000e+00
175    1.300000e-08
177    0.000000e+00
178    0.000000e+00
           ...     
359    1.300000e-08
361    1.500000e-08
365    1.600000e-08
368    5.000000e-09
369    2.400000e-08
380    6.000000e-09
386    2.000000e-09
391    1.000000e-09
395    0.000000e+00
397    0.000000e+00
402    0.000000e+00
403    1.100000e-08
417    0.000000e+00
419    0.000000e+00
425    0.000000e+00
426    0.000000e+00
431    0.000000e+00
432    0.000000e+00
434    0.000000e+00
